# Map with WOW:-P

We will show a classical [map](https://en.wikipedia.org/wiki/Map_%28parallel_pattern%29) operation with wowp, inluding a parallel run using ipyparallel scheduler.

We will use FuncActor as a test actor. Map is inside wowp.actors.mapreduce.

In [1]:
from wowp.actors import FuncActor
from wowp.schedulers import LinearizedScheduler, IPyClusterScheduler
from wowp.actors.mapreduce import Map

In [2]:
# numpy will perform some calculations
import numpy as np

The Map actor takes as the main argument the actor class (i.e. not an instance) and the arguments (positional and keyword) for the actor \_\_init\_\_ function. We will map numpy.sin.

In [3]:
map_actor = Map(
    FuncActor,
    args=(np.sin, ),
    scheduler=LinearizedScheduler())

Now create a sample input array and run the map actor, first with a linear (sequential) scheduler:

In [4]:
inp = np.linspace(0, np.pi, 10)

... and check the results

In [5]:
results = map_actor(inp=inp)
assert np.allclose(np.sin(inp), results['out'])

Now do the same *in parallel* using IPyClusterScheduler (do not forget to **start an ipyparallel cluster**):

In [6]:
map_parallel = Map(
    FuncActor,
    args=(np.sin, ),
    scheduler=IPyClusterScheduler())

In [7]:
results_p = map_parallel(inp=inp)
assert np.allclose(np.sin(inp), results_p['out'])

### Is it really running in parallel?

Simply check with annotating inputs with host and pid.

In [8]:
from wowp.actors.mapreduce import PassWID

PassWID will output an informative dict with host and pid.

In [9]:
# this will be our current process
PassWID()(inp=1)

{'out': {'host': 'fred.localdomain', 'inp': 1, 'pid': 29935}}

In [10]:
pid_act = Map(PassWID, scheduler=LinearizedScheduler())

In [11]:
# With LinearizedScheduler, this will still be the same process.
pid_act(inp=range(3))['out']

({'host': 'fred.localdomain', 'inp': 0, 'pid': 29935},
 {'host': 'fred.localdomain', 'inp': 1, 'pid': 29935},
 {'host': 'fred.localdomain', 'inp': 2, 'pid': 29935})

Let's try IPyClusterScheduler---observe the pid's!

In [12]:
pid_act_parallel = Map(PassWID, scheduler=IPyClusterScheduler())

In [13]:
pid_act_parallel(inp=range(6))['out']

({'host': 'fred.localdomain', 'inp': 0, 'pid': 29611},
 {'host': 'fred.localdomain', 'inp': 1, 'pid': 29612},
 {'host': 'fred.localdomain', 'inp': 2, 'pid': 29610},
 {'host': 'fred.localdomain', 'inp': 3, 'pid': 29611},
 {'host': 'fred.localdomain', 'inp': 4, 'pid': 29612},
 {'host': 'fred.localdomain', 'inp': 5, 'pid': 29610})